In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool

In [ ]:
from pathlib import Path

if Path("/kaggle/input/playground-series-s6e2/").is_dir():
    path = "/kaggle/input/playground-series-s6e2/"
else:
    path = "data/"
train = pd.read_csv(f"{path}train.csv")
test = pd.read_csv(f"{path}test.csv")

In [ ]:
train['y'] = train['Heart Disease'].map({'Presence' : 1, 'Absence' : 0})

In [ ]:
X_test = test
X = train.drop(columns=['Heart Disease', 'y'])
y = train['y']

In [ ]:
folds = 3
param_comb = 2

params = {
    "min_child_weight": [1, 5, 10],
    "gamma": [0.0, 0.5, 1.0, 1.5, 5.0],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "max_depth": [5, 11, 20],
    "learning_rate": [0.0001, 0.0008, 0.01, 0.1, 1.0],
    "reg_alpha": [0.0, 0.1, 0.5, 1.0],
    "reg_lambda": [0.5, 1.0, 3.0, 10.0],
}

xgb_clf = XGBClassifier(
    n_estimators=1000,
    objective="binary:logistic",
    eval_metric="auc",
    tree_method="hist",
    random_state=42,
    n_jobs=1,
)

skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)

random_search = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=params,
    n_iter=param_comb,
    scoring="roc_auc",
    n_jobs=-1,
    cv=skf,
    verbose=3,
    random_state=42,
    return_train_score=True,
)

random_search.fit(X, y)

y_probs_val_rand = random_search.predict_proba(X)[:, 1]